In [4]:

import numpy as np 
import pandas as pd 
import seaborn as sb
import xgboost as xgb


from subprocess import check_output
#print(check_output(["ls", "../input"]).decode("utf8"))

In [7]:
#explore the data


train = pd.read_csv("/home/amal/Bureau/train.csv")
#print(train.head())

test = pd.read_csv("/home/amal/Bureau/test.csv")

In [8]:
#sum features
featuresSum = train.sum()
#print(featuresSum)
#print(type(train))
#plot the sum of features in desceding order
featuresSum.drop(['target', 'id']).order().plot(kind='barh', figsize=(8,16))

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:6: FutureWarning: order is deprecated, use sort_values(...)


In [9]:
#Clases are String like "Class_1" => we map them as numbers from 0-8 (9 classes)

class_range = range(1, 10)
class_dict = {}

for n in class_range:
    class_dict['Class_{}'.format(n)] = n-1

#print(class_dict)
#print(train.head())
train['target'] = train['target'].map(class_dict)
#print(train['target'].head())

#next we plot the count for each class unsing seaborn libary
sb.countplot(x='target', data= train)

In [11]:
#next we create training and testing sets
X_train = train.drop(["id", "target"], axis=1)
Y_train = train["target"].copy()
X_test = test.drop("id", axis = 1).copy()

In [12]:
# we use the training and test sets to make a prediction based on Boosted Trees => library XgBoost
# http://xgboost.readthedocs.io/en/latest/model.html

params = {"objective": "multi:softprob", "eval_metric":"mlogloss", "num_class": 9}

T_train_xgb = xgb.DMatrix(X_train, Y_train)
X_test_xgb  = xgb.DMatrix(X_test)

#number of iterations - initial 20, 
gbm = xgb.train(params, T_train_xgb, 50)
Y_pred = gbm.predict(X_test_xgb)

In [14]:
#Make a submission based on model "sample_submission.csv"

submission = pd.DataFrame({ "id": test["id"]})

i = 0

for num in class_range:
    col_name = str("Class_{}".format(num))
    submission[col_name] = Y_pred[:,i]
    i = i + 1
    
submission.to_csv('/home/amal/Bureau/otto1.csv', index=False)